# 🔥 _DeepFire_: API Project for Fire Detection 🔥

In this project, you'll apply your skills at neural network development in a new way: taking a model that you've trained yourself and deploying it to a static webpage that you can work with to upload new images and get prediction accuracy results. 

This project will primarily focus on your abilities in creating and testing neural network architecture development. 

Boilerplate and supporting architectures have been provided for a multitude of tasks ranging from data preprocessing, processing, ingestion, and predictive assessment – however, major tasks and design work will ultimately be left to you to approach and figure out ideal, optimized solutions. 

#### 🔹 General Importations

As always, we'll start with importing basic tools and functions for our task.

In [3]:
import numpy as np
import matplotlib.pyplot as plt

import os, PIL
from glob import glob

import tensorflow as tf

print(tf.__version__)

2.6.0


---

#### 🔎 Initializing Deep Learning Tools 🔍

---

Your first task will be crucial to ensuring the successful implementation of the rest of your notebook. 

**Initialize each line with the correct function type from the TensorFlow documentation.**

Feel free to refer throughout the notebook and across previous notebooks to see which TensorFlow architectures you've used for similar tasks. 

To give you a guide for how this should look, you've been provided with a single correct function declaration in the form of `image_dataset_from_directory` at the end of the cell. 

---

In [31]:
""" Sequential Model Architecture """
# TODO: Initialize the sequential model architecture here.
Sequential = tf.keras.models.Sequential

""" Data Preprocessing Functions """
# TODO: Initialize the experimental resizing layer here.
Resizing = tf.keras.layers.experimental.preprocessing.Resizing
# TODO: Initialize the experimental rescaling layer here.
Rescaling = tf.keras.layers.experimental.preprocessing.Rescaling

""" Data Augmentation Functions """
# TODO: Initialize the experimental random flipping layer here.
RandomFlip = tf.keras.layers.experimental.preprocessing.RandomFlip
# TODO: Initialize the experimental random rotating layer here.
RandomRotation = tf.keras.layers.experimental.preprocessing.RandomRotation
# TODO: Initialize the experimental random zooming layer here.
RandomZoom = tf.keras.layers.experimental.preprocessing.RandomZoom

""" Artificial Neural Network Layer Inventory """
# TODO: Initialize the dense connective layer here.
Dense = tf.keras.layers.Dense
# TODO: Initialize the dropout regularization layer here.
Dropout = tf.keras.layers.Dropout

""" Convolutional Neural Network Layer Inventory """
# TODO: Initialize the 2D convolutional layer here.
Conv2D = tf.keras.layers.Conv2D
# TODO: Initialize the 2D max pooling layer here.
MaxPool2D = tf.keras.layers.MaxPool2D
# TODO: Initialize the flattening layer here.
Flatten = tf.keras.layers.Flatten

""" Residual Network Layer Inventory """
# TODO: Initialize the Residual Network multilayer model here.
# TODO: Make sure you initialize the 50-layer residual network! 
# HINT: Look up `ResNet50` for appropriate documentation. 
ResNet50 = tf.keras.applications.resnet50.ResNet50

""" Function to Load Images from Target Folder """
image_dataset_from_directory = tf.keras.preprocessing.image_dataset_from_directory

#### 🔹 Precheck Image Dataset Sizes



In [8]:
# Use the `glob.glob` function to show how many images are in each folder
DATA_DIRECTORY = "../dataset/Images/"
FIRE_IMAGES_PATTERN = f"{DATA_DIRECTORY}/Fire_Images/*"
NOT_FIRE_IMAGES_PATTERN = f"{DATA_DIRECTORY}/Normal_Images/*"

print(f"Number of fire image samples: {len(glob(FIRE_IMAGES_PATTERN))}")
print(f"Number of non-fire image samples: {len(glob(NOT_FIRE_IMAGES_PATTERN))}")

Number of fire image samples: 109
Number of non-fire image samples: 532


#### Load Dataset

In [17]:
batch_size = 32
IMAGE_HEIGHT = IMAGE_WIDTH = 256

In [18]:
train = image_dataset_from_directory(
    directory=DATA_DIRECTORY,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=batch_size
)

Found 641 files belonging to 2 classes.
Using 513 files for training.


In [19]:
validation = image_dataset_from_directory(
    directory=DATA_DIRECTORY,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=batch_size
)

Found 641 files belonging to 2 classes.
Using 128 files for validation.


In [20]:
class_names = train.class_names

In [21]:
def configure_performant_datasets(dataset, shuffling=None):
    """ 
    Custom function to prefetch and cache stored elements
    of retrieved image data to boost latency and performance
    at the cost of higher memory usage. 
    """
    AUTOTUNE = tf.data.AUTOTUNE
    # Cache and prefetch elements of input data for boosted performance
    if not shuffling:
        return dataset.cache().prefetch(buffer_size=AUTOTUNE)
    else:
        return dataset.cache().shuffle(shuffling).prefetch(buffer_size=AUTOTUNE)

In [22]:
train =         configure_performant_datasets(train, shuffling=1000)
validation =    configure_performant_datasets(validation)

In [30]:
resizing_layer =        Resizing(IMAGE_HEIGHT,
                                 IMAGE_WIDTH)

normalization_layer =   Rescaling(1./255,
                                  input_shape=(IMAGE_HEIGHT,
                                               IMAGE_WIDTH,
                                               3))